<a href="https://colab.research.google.com/github/ashi-agrawal-06/Python-bot-for-Amazon-data-scraping/blob/main/amazon_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import requests
import numpy as np
from bs4 import BeautifulSoup as bp
import time

In [ ]:
h={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}

In [ ]:
url='https://www.amazon.in/s?k=mobiles&page='

In [ ]:
def parse_phone_page(doc,h):
  dic=[]
  for i in doc.find_all('div',class_='s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col sg-col-12-of-16'):
    link="https://www.amazon.in"+i.find('a',class_='a-link-normal a-text-normal')['href']
    sp=parse_content(link,h)
    dic.append(sp)
  for i in doc.find_all('div',class_='s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 AdHolder sg-col sg-col-12-of-16'):
    link="https://www.amazon.in"+i.find('a',class_='a-link-normal a-text-normal')['href']
    sp=parse_content(link,h)
    dic.append(sp)
  return dic





def parse_content(link,h):
  re=requests.get(link,headers=h)
  doc1=bp(re.text,'lxml')
  try:
   url2="https://www.amazon.in"+doc1.find('a',attrs={'data-hook':'see-all-reviews-link-foot'})['href']+"&pageNumber="
   rev=extract_reviews(url2,h)
  except Exception as ee:
    rev= None
    #print(link,'rev')
  spec={}
  t=doc1.find('table',attrs={'id':'productDetails_techSpec_section_1'})
  for i in t.find_all('tr'):
    sp=i.find('th').text.strip()
    val=i.find('td').text.strip()
    spec.update({sp:val})
  q=doc1.find('table',attrs={'id':'productDetails_detailBullets_sections1'})
  for i in q.find_all('tr'):
    sp=i.find('th').text.strip()
    val=i.find('td').text.strip()
    spec.update({sp:val})
  
  for i in doc1.find_all('th',class_='attribute-heading'):
    try:
      spec.update({i.find('span',class_='a-size-base').text:i.find_next_sibling('td').text.strip()})
    except Exception as e3:
      print(link)
      continue
  name=doc1.find('span',attrs={'id':'productTitle'}).text.strip()
  try:
    pr=doc1.find('td',class_='a-span12')
    if pr:
      price=pr.find('span').text.split('₹')[1].strip()
    else:
      price=doc1.find('span',class_='a-size-base a-color-price').text.split('₹')[1].strip()
    
  except Exception as e:
    print(link)
    price= None
  try:
    stars=doc1.find('span',class_='a-icon-alt').text.split(' ')[0]
  except Exception as e1:
    stars= None
  spec.update({'Name':name,'Detailed Reviews':rev,'Price':price,'Rating': stars})
  return spec




def extract_reviews(url2_c,h):  
  reviews=[]
  for i in range(1,6):
    req1=requests.get(f'{url2_c}{i}',headers=h)
    doc3=bp(req1.text)
    for i in doc3.find_all('div',class_='a-section review aok-relative'):
        review=i.find('span',class_='a-size-base review-text review-text-content').text.strip()
        reviews.append(review)
  return reviews


In [ ]:
df=pd.DataFrame()
for i in range(1,2):
  url1=f'{url}{i}'
  r=requests.get(url1,headers=h)
  doc=bp(r.text,'lxml')
  sp=parse_phone_page(doc,h)
  dt=pd.DataFrame(sp)
  df=df.append(dt,True)
df.to_csv('Amazon_mobiles.csv')

In [ ]:
df1=df[list(set(df.columns))]

In [ ]:
df1 = df.loc[:,~df.columns.duplicated()]

In [ ]:
len(df1.columns)

53

In [ ]:
df1.columns

Index(['OS', 'RAM', 'Product Dimensions', 'Batteries', 'Item model number',
       'Wireless communication technologies', 'Connectivity technologies',
       'Special features', 'Display technology', 'Other display features',
       'Device interface - primary', 'Other camera features', 'Form factor',
       'Colour', 'Battery Power Rating', 'Whats in the box', 'Manufacturer',
       'Item Weight', 'ASIN', 'Customer Reviews', 'Best Sellers Rank',
       'Date First Available', 'Item Dimensions LxWxH', 'Generic Name',
       'Rear Camera Lens 1', 'Rear Camera Lens 2', 'Screen Size',
       'Screen Type', 'Battery Power (In mAH)', 'Inbuilt Storage (in GB)',
       'Expandable Storage', 'Processor Brand', 'Operating System',
       'Warranty Details', 'Battery type', 'Processor Speed',
       'What's in the box', 'Additional Features', 'Item Dimensions', 'Name',
       'Detailed Reviews', 'Price', 'Rating', 'Country of Origin',
       'Resolution', 'Phone Talk Time', 'Phone Standby Time (

In [ ]:
len(set(df1.columns))

53

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 53 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   OS                                   16 non-null     object
 1   RAM                                  16 non-null     object
 2   Product Dimensions                   15 non-null     object
 3   Batteries                            14 non-null     object
 4   Item model number                    16 non-null     object
 5   Wireless communication technologies  15 non-null     object
 6   Connectivity technologies            16 non-null     object
 7   Special features                     16 non-null     object
 8   Display technology                   13 non-null     object
 9   Other display features               16 non-null     object
 10  Device interface - primary           12 non-null     object
 11  Other camera features                16 non-nul